# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
from db_connection.connection import PostgresConnection

# Add the src directory to the system path


In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from customer_satisfaction_analysis import (
      load_data,calculate_scores, calculate_satisfaction_score, 
    regression_model, kmeans_clustering, aggregate_clusters, export_to_mysql, model_deployment_tracking
)

## Initialize and connect to the PostgreSQL database


In [3]:
db = PostgresConnection()
db.connect()

Connected to PostgreSQL Database!


# Load the data


In [4]:
query = "SELECT * FROM xdr_data;"
user_data = db.fetch_data(query)

# Display the first few rows of the data
user_data.head()

c:\Users\habteyes.asfaw\10Accadamy\TellCo-Telecom-User-Analytics\db_connection\connection.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


# Task 4.1:Calculate Engagement and Experience Scores


In [5]:
print(user_data.columns)


Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [10]:
scored_data = calculate_scores(user_data)


In [11]:
# Display the first few rows of the data with scores
print("Data with calculated scores:")
print(scored_data.head())

Data with calculated scores:
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Netflix UL (Bytes)  Gaming DL (Bytes)  \
0  9.16456699548519E+015  ...           9656251.0        278082303.0   
1                L77566A  ...          172271

# Task 4.2: Calculate satisfaction score and get top 10 satisfied customers


In [12]:
scored_data, top_10_customers = calculate_satisfaction_score(scored_data)
print("Top 10 satisfied customers:")
print(top_10_customers)


Top 10 satisfied customers:
           Bearer Id            Start  Start ms             End  End ms  \
92379   7.277826e+18  4/27/2019 15:19      28.0  4/28/2019 0:16   805.0   
5926    1.304243e+19  4/24/2019 10:59     973.0  4/25/2019 1:20   383.0   
137168  1.304243e+19   4/29/2019 1:31     845.0  4/30/2019 1:31   729.0   
116895  1.304243e+19   4/28/2019 1:44     894.0  4/29/2019 1:44   862.0   
99297   7.277826e+18  4/27/2019 17:19     641.0  4/28/2019 1:31   202.0   
91313   7.277826e+18  4/27/2019 14:46     507.0  4/28/2019 3:54   281.0   
45409   7.277826e+18   4/25/2019 6:51     103.0  4/26/2019 0:50   718.0   
9283    7.277826e+18  4/24/2019 16:06      19.0  4/25/2019 0:06   859.0   
91621   7.277826e+18  4/27/2019 14:56     742.0  4/28/2019 0:01   667.0   
124763  1.304243e+19   4/28/2019 6:15     135.0  4/29/2019 6:15   129.0   

        Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
92379     32245.0  2.082019e+14   3.361508e+10  8.683890e+13   
5926      51676.0 

# Task 4.3: Build Regression Model to Predict Satisfaction Score


In [13]:
model, mse = regression_model(scored_data)
print(f"Mean Squared Error of the regression model: {mse}")

Mean Squared Error of the regression model: 1.998355754279054e-21
